# Microsimulation with the `humanleague` package

The `humanleague` package contains implementations of iterative propotional fitting and quasirandom integer without-replacement sampling algorithms.

We'll begin by following the first stage of an example of population generation set out in "Microsimulation for Demography" (Lomax and Smith, 2017). The aim is to generate a representative population of 254,096 people who each have the following properties: MSOA (area; see below), sex, age in years, and ethnicity. In the paper, the generated population is then used to perform a dynamic microsimulation of the population through time (fertility and mortality rates are applied based on individuals' characterstics). Here, we'll focus on the first, static, microsimulation where the population is generated from two datasets that cannot be directly combined.

To do this, we will combine the two datasets from the 2011 census using both iterative proportional fitting and quasirandom integer sampling. The `humanleague` package implements these algorithms in C++, which can be accessed through either Python or R.

## Methods used in this notebook

### Quasirandom integer without-replacement sampling

### Iterative proportional fitting

## Tower Hamlets example

In [ ]:
import humanleague
import numpy as np
import os
import pandas as pd
import urllib.request

The two datasets we use here contain data from MSOAs within the London Borough of Tower Hamlets. An MSOA is a Middle-layer Super Output Area - a region that has a population of approximately 7000 people. An MSOA is one of several output areas in the following heirarchy:
- Output Area, OA, average population ~300
- Lower-layer Super Output Area, LSOA, population ~1000-3000
- Middle-layer Super Output Area, MSOA, population ~5000-15,000

The data we use here come from two tables from the 2011 UK Census:
- DC2101EW: Ethnic group by sex and age band
- DC117EW: Sex by age in years

The files we use here are preprocessed versions of the above, taken from the GitHub repository associated with Lomax and Smith (2017).

### Load the data

First, let's check whether the data files are already available, and download them if not.

In [ ]:
current_dir = %pwd
data_dir = os.path.join(current_dir, "..", "..", "datasets", "tower-hamlets")
if not os.path.isdir(data_dir):
    print("Tower Hamlets data folder is not present; creating...")
    os.mkdir(data_dir)
    
if not os.path.exists(os.path.join(data_dir, "sexAgeEth.csv")):
    print("Downloading sexAgeEth.csv...")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/virgesmith/demographyMicrosim/master/data/sexAgeEth.csv",
                               os.path.join(data_dir, "sexAgeEth.csv"))
    
if not os.path.exists(os.path.join(data_dir, "sexAgeYear.csv")):
    print("Downloading sexAgeYear.csv...")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/virgesmith/demographyMicrosim/master/data/sexAgeYear.csv",
                               os.path.join(data_dir, "sexAgeYear.csv"))

Our first set of constraints are given in the "ageSexEth.csv" file. This file contains counts of the numbers of people who are resident in each MSOA by their sex, age band and ethnicity.

In [ ]:
con_ethBySexAgeBand = pd.read_csv(os.path.join(data_dir, "sexAgeEth.csv"), sep=";")
con_ethBySexAgeBand.head()

We also have a second set of contstraints, which we load from the "sexAgeYear.csv" file. These records are lacking an ethnicity attribute, but the exact age is known.

In [ ]:
con_ageBySex = pd.read_csv(os.path.join(data_dir, "sexAgeYear.csv"), sep=";")
con_ageBySex.head()

We'll need a way to convert between the exact age and the age bands, so let's define a utility function to do so here:

In [ ]:
def get_age_bounds(age_band):
    ab_limits = age_band.split("-")
            
    if len(ab_limits) == 2:
        lower_age = int(ab_limits[0])
        upper_age = int(ab_limits[1])
    elif len(ab_limits) == 1:
        if "+" in ab_limits[0]:
            lower_age = int(ab_limits[0].strip("+"))
            upper_age = 999
        else:
            lower_age = int(ab_limits[0])
            upper_age = int(ab_limits[0])  
    else:
        raise Exception("Cannot work out upper and lower limits for age")
        
    return {"lower": lower_age, "upper": upper_age}

Let's also identify the unique values of MSOA, sex and age band. We'll need to perform the microsimulation for each of these stages separately.

In [ ]:
msoa = con_ethBySexAgeBand["MSOA"].unique()
sex = con_ethBySexAgeBand["Sex"].unique()
ageband = con_ethBySexAgeBand["AgeBand"].unique()

### Perform microsimulation with QIS

The code below is based very closely on the code from Lomax and Smith (2017). There have been some major changes to the functions available in the `humanleague` package since the paper was released, which should make the following steps less complicated - but for now, we'll stick to the original method as closely as possible to reduce potential error.

(To do: remove some of the loops to use `qis` as intended, and compare.)

In [ ]:
qis_synthetic = pd.DataFrame(columns=["MSOA", "Age", "Sex", "Ethnicity", "Persons"])

for m in msoa:
    for s in sex:
        for ab in ageband:
            
            age_limits = get_age_bounds(ab)
            
            # Find the marginal labels: these are the properties for which we will generate a population in this loop
            ml_eth = con_ethBySexAgeBand.loc[(con_ethBySexAgeBand["MSOA"] == m) &
                                             (con_ethBySexAgeBand["Sex"] == s) &
                                             (con_ethBySexAgeBand["AgeBand"] == ab)]["Ethnicity"]
            ml_age = con_ageBySex.loc[(con_ageBySex["MSOA"] == m) &
                                      (con_ageBySex["Sex"] == s) &
                                      (con_ageBySex["Age"] >= age_limits["lower"]) &
                                      (con_ageBySex["Age"] <= age_limits["upper"])]["Age"]
            
            # Now find the marginal frequencies: these are the numbers of people with these properties
            mf_eth = con_ethBySexAgeBand.loc[(con_ethBySexAgeBand["MSOA"] == m) &
                                             (con_ethBySexAgeBand["Sex"] == s) &
                                             (con_ethBySexAgeBand["AgeBand"] == ab)]["Persons"]
            mf_age = con_ageBySex.loc[(con_ageBySex["MSOA"] == m) &
                                      (con_ageBySex["Sex"] == s) &
                                      (con_ageBySex["Age"] >= age_limits["lower"]) &
                                      (con_ageBySex["Age"] <= age_limits["upper"])]["Persons"]            
            
            # Only perform the microsim step if there are some records with this combination of MSOA, sex and eth
            if sum(mf_eth) > 0:
                # The result that we're given back here contains frequencies
                # for each combination of age (column) and ethnicity (row)
                res = humanleague.qis([np.array([0]), np.array([1])], [mf_eth.to_numpy(), mf_age.to_numpy()])
                
                # Reorganise the output, add MSOA and Sex and rearrange to match format of storage dataframe
                synth = pd.DataFrame.from_records(res["result"], index=ml_eth, columns=ml_age)
                synth = synth.unstack().reset_index()
                synth.columns.values[2] = "Persons"
                synth["MSOA"] = m
                synth["Sex"] = s
                synth = synth[qis_synthetic.columns.values]
                
                # Append the synthetic data for this MSOA/sex combination to the storage dataframe
                qis_synthetic = qis_synthetic.append(synth)

### Perform microsimulation with IPF